In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/wordnet/wordnet/data.adj
/kaggle/input/wordnet/wordnet/LICENSE
/kaggle/input/wordnet/wordnet/index.adj
/kaggle/input/wordnet/wordnet/README
/kaggle/input/wordnet/wordnet/verb.exc
/kaggle/input/wordnet/wordnet/index.adv
/kaggle/input/wordnet/wordnet/index.verb
/kaggle/input/wordnet/wordnet/data.verb
/kaggle/input/wordnet/wordnet/lexnames
/kaggle/input/wordnet/wordnet/data.noun
/kaggle/input/wordnet/wordnet/noun.exc
/kaggle/input/wordnet/wordnet/index.noun
/kaggle/input/wordnet/wordnet/adj.exc
/kaggle/input/wordnet/wordnet/index.sense
/kaggle/input/wordnet/wordnet/adv.exc
/kaggle/input/wordnet/wordnet/cntlist.rev
/kaggle/input/wordnet/wordnet/data.adv
/kaggle/input/wordnet/wordnet/citation.bib
/kaggle/input/ml-challenge-2023-training-data/train.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import warnings
warnings.filterwarnings("ignore")

import nltk
import re
import string
from collections import Counter
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [3]:
print(pd.__version__)

2.2.2


In [26]:
import shutil
import os

# Paths
source_dir = '/kaggle/input/wordnet/wordnet'  # Adjust this based on actual paths
destination_dir = '/kaggle/working/nltk_data/corpora/wordnet'

# Ensure the destination directory exists
os.makedirs(destination_dir, exist_ok=True)

# Move files to the destination directory
for item in os.listdir(source_dir):
    source_path = os.path.join(source_dir, item)
    destination_path = os.path.join(destination_dir, item)
    if os.path.isdir(source_path):
        shutil.copytree(source_path, destination_path, dirs_exist_ok=True)
    else:
        shutil.copy2(source_path, destination_path)

print("Files moved successfully.")


Files moved successfully.


In [28]:
import nltk

# Add the path to NLTK's data path
nltk.data.path.append('/kaggle/working/nltk_data')

# Verify the path
print("NLTK data paths:", nltk.data.path)

# Check if WordNet is available
try:
    nltk.data.find('corpora/wordnet')
    print("WordNet is installed and found.")
except LookupError:
    print("WordNet is not installed.")


NLTK data paths: ['/root/nltk_data', '/usr/share/nltk_data', '/usr/local/share/nltk_data', '/usr/lib/nltk_data', '/usr/local/lib/nltk_data', '/kaggle/working/nltk_data']
WordNet is installed and found.


In [29]:
from nltk.corpus import wordnet

# Test WordNet functionality
synsets = wordnet.synsets('example')
print(f"Synsets for 'example': {synsets}")

# Print some details about the first synset
if synsets:
    print(f"Definition: {synsets[0].definition()}")
    print(f"Examples: {synsets[0].examples()}")
else:
    print("No synsets found.")


Synsets for 'example': [Synset('example.n.01'), Synset('model.n.07'), Synset('exemplar.n.01'), Synset('example.n.04'), Synset('case.n.01'), Synset('exercise.n.04')]
Definition: an item of information that is typical of a class or group
Examples: ['this patient provides a typical example of the syndrome', 'there is an example on page 10']


In [30]:
import os
import nltk

# Clear any unnecessary paths
nltk.data.path = ['/kaggle/working/nltk_data']  # Ensure it points to the right directory

# Verify again
print("NLTK data paths after cleanup:", nltk.data.path)

# Check if WordNet is accessible
try:
    nltk.data.find('corpora/wordnet')
    print("WordNet is successfully accessible.")
except LookupError:
    print("Failed to access WordNet.")


NLTK data paths after cleanup: ['/kaggle/working/nltk_data']
WordNet is successfully accessible.


In [4]:
df = pd.read_csv("/kaggle/input/ml-challenge-2023-training-data/train.csv", encoding="utf-8")


In [5]:
df.head()

,PRODUCT_ID,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH
0,1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000
1,2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000
2,2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495
3,1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574
4,283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000


In [6]:
df.shape

(2249698, 6)

In [7]:
df['BULLET_POINTS'].iloc[0]

'[LUXURIOUS & APPEALING: Beautiful custom-made curtains to decorate any home or office | Includes inbuilt tieback to hold the curtain | Completely finished and ready to hang on walls & windows,MATERIAL: Luxurious & versatile fabric with a natural finish | High colour fastness | State-of-the-art digital printing ensures colour consistency and prevents any fading | Eyelets; Cotton Canvas; Width 4.5feet (54inch) | Multicolour | PACKAGE: 2 Room Curtains Eyelets | SIZE: Height 5 feet (60 inch); SET OF 2 PCS,BLACKOUT CURTAIN: 100% opaque & heavy premium cotton canvas fabric | Tight knitted, long life & durable fabric | Printing only on front side with a plain colour back side,MADE TO PERFECTION: Large eyelets at the top to put hanging hooks | Perfectly tailored seams for durability | Refined stitching with a matching thread color,QUALITY ASSURED: Gentle wash with similar colors in cold water | Avoid direct sunlight to prevent fading | Dispatched after MULTIPLE QUALITY CHECKS]'

In [8]:
df["PRODUCT_ID"].nunique()

2249698

In [9]:
df.shape

(2249698, 6)

In [10]:
## We can make PRODUCT_ID as index
df.set_index('PRODUCT_ID', inplace=True)

In [11]:
df.index
## we updated the indexes of dataframe

Index([1925202, 2673191, 2765088, 1594019,  283658, 2152929,  413758, 2026580,
       2050239, 2998633,
       ...
        575288, 1616561, 1096345,  571566, 2690647, 2422167, 2766635, 1987786,
       1165754, 1072666],
      dtype='int64', name='PRODUCT_ID', length=2249698)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2249698 entries, 1925202 to 1072666
Data columns (total 5 columns):
 #   Column           Dtype  
---  ------           -----  
 0   TITLE            object 
 1   BULLET_POINTS    object 
 2   DESCRIPTION      object 
 3   PRODUCT_TYPE_ID  int64  
 4   PRODUCT_LENGTH   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 103.0+ MB


In [13]:
def df_information(data):
  shape         = data.shape
  info          = data.info()
  describe      = data.describe()
  column_name   = data.columns
  null_values   = data.isnull().sum()

  return shape, info, describe, column_name, null_values

In [14]:
df.isna().sum()

TITLE                   13
BULLET_POINTS       837366
DESCRIPTION        1157382
PRODUCT_TYPE_ID          0
PRODUCT_LENGTH           0
dtype: int64

In [15]:
## Joining the columns which are the inputs
new_df = df.copy()
# need to fill the nan values else it gonna give error
new_df["inputs"] = df[["TITLE", "BULLET_POINTS", "DESCRIPTION"]].fillna(" ").apply(lambda x: ' '.join(x), axis=1)

In [16]:
new_df.head()

,TITLE,BULLET_POINTS,DESCRIPTION,PRODUCT_TYPE_ID,PRODUCT_LENGTH,inputs
PRODUCT_ID,,,,,,
1925202,ArtzFolio Tulip Flowers Blackout Curtain for D...,[LUXURIOUS & APPEALING: Beautiful custom-made ...,NaN,1650,2125.980000,ArtzFolio Tulip Flowers Blackout Curtain for D...
2673191,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...,"[Harry Potter Hedwig Pyjamas (6-16 Yrs),100% c...",NaN,2755,393.700000,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...
2765088,PRIKNIK Horn Red Electric Air Horn Compressor ...,"[Loud Dual Tone Trumpet Horn, Compatible With ...","Specifications: Color: Red, Material: Aluminiu...",7537,748.031495,PRIKNIK Horn Red Electric Air Horn Compressor ...
1594019,ALISHAH Women's Cotton Ankle Length Leggings C...,[Made By 95%cotton and 5% Lycra which gives yo...,AISHAH Women's Lycra Cotton Ankel Leggings. Br...,2996,787.401574,ALISHAH Women's Cotton Ankle Length Leggings C...
283658,The United Empire Loyalists: A Chronicle of th...,NaN,NaN,6112,598.424000,The United Empire Loyalists: A Chronicle of th...


In [17]:
new_df = new_df.drop(["TITLE", "BULLET_POINTS", "DESCRIPTION", "PRODUCT_TYPE_ID"], axis=1)

In [18]:
new_df.head()

,PRODUCT_LENGTH,inputs
PRODUCT_ID,,
1925202,2125.980000,ArtzFolio Tulip Flowers Blackout Curtain for D...
2673191,393.700000,Marks & Spencer Girls' Pyjama Sets T86_2561C_N...
2765088,748.031495,PRIKNIK Horn Red Electric Air Horn Compressor ...
1594019,787.401574,ALISHAH Women's Cotton Ankle Length Leggings C...
283658,598.424000,The United Empire Loyalists: A Chronicle of th...


In [19]:
new_df.dropna(axis=0, inplace=True)

In [20]:
new_df.isna().sum()

PRODUCT_LENGTH    0
inputs            0
dtype: int64

In [21]:
new_df.shape

(2249698, 2)

In [22]:
nltk.download("stopwords")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [23]:
def remove_emojis_and_symbols(text):
    # This regex targets most emojis, hearts, and various symbols
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        u"\U0001F680-\U0001F6FF"  # Transport & Map symbols
        u"\U0001F1E0-\U0001F1FF"  # Flags (iOS)
        u"\U00002702-\U000027B0"  # Dingbats
        u"\U000024C2-\U0001F251"  # Enclosed characters
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols (includes hearts)
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        "]+", flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

In [24]:
# will start preprocessing the inputs
def preprocessing(data):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words("english"))

    def clean_sentence(sentence):
        # Convert to lowercase
        sentence = sentence.lower()

        # Remove non-alphanumeric characters
        sentence = re.sub("[^a-z0-9]", " ", sentence)

        # Remove URLs
        sentence = re.sub(r'(http|https|ftp|ssh)://[\w_-]+(?:\.[\w_-]+)+[\w.,@?^=%&/~+#-]*[\w@?^=%&/~+#-]?', '', sentence)

        # removing emojis and other symobls that are not cleaned
        sentence = remove_emojis_and_symbols(sentence)

        # Remove stopwords
        sentence = " ".join([word for word in sentence.split() if word not in stop_words])

        # Clean any potential HTML tags
        sentence = BeautifulSoup(sentence, "lxml").get_text()

        # Lemmatize words
        sentence = " ".join([lemmatizer.lemmatize(word) for word in sentence.split()])

        return sentence

    # Apply the cleaning function to each row
    data['inputs'] = data['inputs'].apply(clean_sentence)

    return data

In [ ]:
cleaned_df     = preprocessing(new_df)

In [ ]:
cleaned_df.head()

In [ ]:
cleaned_df.shape

In [ ]:
def vocabulary(df):
    words = [word for sentence in df.inputs for word in sentence.split()]
    word_counts = Counter(words)
    unique_words = set(word_counts)
    sorted_words = word_counts.most_common()

    return sorted_words, unique_words

In [ ]:
word_counts, unique_words = vocabulary(cleaned_df)

In [ ]:
# word_counts

In [ ]:
len(unique_words)

In [ ]:
words = []
for i, (w, c) in enumerate(word_counts):
  if c < 5:
    words.append(w)

In [ ]:
len(words)

In [ ]:
def remove_rare_wors(word_counts):
  word_list = []
  for i, (word, cout) in enumerate(word_counts):
      if word.isdigit():
        continue
      elif cout > 5:
        word_list.append(word)

  return word_list

In [ ]:
clenaed_vocabulary_   = remove_rare_wors(word_counts)

In [ ]:
type(45)

In [ ]:
len(clenaed_vocabulary_)

In [ ]:
# clenaed_vocabulary_

In [ ]:
def word_to_int(clenaed_vocabulary_):
  word_to_numeric = {}
  for i, word in enumerate(clenaed_vocabulary_):
    word_to_numeric[word] = i + 1

  return word_to_numeric

In [ ]:
word_to_numeric   = word_to_int(clenaed_vocabulary_)

In [ ]:
# word_to_numeric

In [ ]:
def sentence_to_int(data):
  data["inputs"] = data["inputs"].apply(lambda x: [word_to_numeric[word] for word in x.split() if word in clenaed_vocabulary_])

  return data

In [ ]:
cleaned_df_to_numeric = sentence_to_int(cleaned_df)

In [ ]:
cleaned_df_to_numeric["length"] = cleaned_df_to_numeric["inputs"].apply(lambda x: len(x))

In [ ]:
cleaned_df_to_numeric.head()

In [ ]:
cleaned_df_to_numeric.describe()

In [ ]:
cleaned_df_to_numeric.shape

In [ ]:
def boxplot(data):
  plt.figure(figsize=(14, 6))
  sns.boxplot(data)
  plt.show()

In [ ]:
boxplot(cleaned_df_to_numeric["length"])

In [ ]:
len(cleaned_df_to_numeric[cleaned_df_to_numeric["length"] >= 280])

In [ ]:
len(cleaned_df_to_numeric[cleaned_df_to_numeric["length"] <= 6])

In [ ]:
len(cleaned_df_to_numeric[cleaned_df_to_numeric["length"] == 0 ])

In [ ]:
## remvoing the rows from the dataset where

def remove_outliers(data, col):

  Q1      = data[col].quantile(0.25)
  Q3      = data[col].quantile(0.75)
  IQR     = Q3 - Q1

  lower_bound = Q1 -1.5 * IQR
  upper_bound = Q3 + 1.5 * IQR

  data = data[(data[col] >= lower_bound) & (data[col] <= upper_bound) & data[col] == 0]

  return data


In [ ]:
cleaned_numeric_data   = remove_outliers(cleaned_df_to_numeric, "length")

In [ ]:
boxplot(cleaned_numeric_data["length"])

In [ ]:
cleaned_numeric_data2   = remove_outliers(cleaned_numeric_data, "length")

In [ ]:
boxplot(cleaned_numeric_data2["length"])

In [ ]:
cleaned_numeric_data2.describe()

In [ ]:
def padding(sentences, seq_length=max(cleaned_numeric_data2["length"])):
  features = np.zeros((len(sentences), seq_length), dtype=int)

  for i, row in enumerate(sentences):
    if len(row) > 0:
      features[i, -len(row):] = np.array(row)[:seq_length]

  return features

In [ ]:
padded_trained_data = padding(cleaned_numeric_data2["inputs"])

In [ ]:
padded_trained_data[0]

In [ ]:
def np_label(label_data):
  labels = np.array(label_data)

  return labels

In [ ]:
cleaned_numeric_data2.columns

In [ ]:
train_np_labels = np_label(cleaned_numeric_data2['PRODUCT_LENGTH'])

In [ ]:
def tensor_dataset(data, labels):
  tensor_data = torch.from_numpy(data)
  tensor_labels = torch.from_numpy(labels)

  tensor_dataset = TensorDataset(tensor_data, tensor_labels)

  return tensor_dataset


In [ ]:
def dataloader(dataset, batch_size):
  dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

  return dataloader

In [ ]:
batch_size = 64
training_dataset = tensor_dataset(padded_trained_data, train_np_labels)
train_loader      = dataloader(training_dataset, batch_size=batch_size)

**Building A LSTM Model**

In [ ]:
class LSTM_Model(nn.Module):
    def __init__(self, input_dim, hidden_dim, embedding_dim, hidden_layers):
        super(LSTM_Model, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.embedding_dim = embedding_dim
        self.hidden_layers = hidden_layers

        # Embedding layer
        self.embedding_layer = nn.Embedding(self.input_dim, self.embedding_dim)

        # LSTM layer
        self.lstm = nn.LSTM(self.embedding_dim,
                            self.hidden_dim,
                            self.hidden_layers,
                            batch_first=True,
                            dropout=0.3)

        # Fully connected layer | output layer
        self.output_layer = nn.Linear(self.hidden_dim, 1)

        # Dropout for regularization
        self.dropout = nn.Dropout(0.3)

    def forward(self, inputs, hidden_inputs, batch_size):
        # Pass inputs through the embedding layer
        embed_outputs = self.embedding_layer(inputs)

        # Pass embeddings through the LSTM layer
        lstm_output, hidden_outputs = self.lstm(embed_outputs, hidden_inputs)

        # Reshape LSTM output for the fully connected layer
        lstm_output_cont = lstm_output[:, -1, :]  

        # Apply dropout
        output = self.dropout(lstm_output_cont)

        # Pass through the output layer (fully connected)
        output = self.output_layer(output)

        # Reshape the output to the correct shape (batch_size, -1)
        output = output.view(batch_size, -1)

        return output, hidden_outputs

In [ ]:
input_dim = len(clenaed_vocabulary_) + 1
hidden_dim = 512
embedding_dim = 50
hidden_layers = 3

model = LSTM_Model(input_dim, hidden_dim, embedding_dim, hidden_layers)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fun = nn.MSELoss()

In [ ]:
def calculate_r2_score(actual_labels, pred_labels):
    # Convert to float for numerical stability
    actual_labels = actual_labels.float()
    pred_labels = pred_labels.float()

    # Calculate the mean of actual labels
    mean_actual = torch.mean(actual_labels)

    # Sum of Squared Errors (SSE)
    sse = torch.sum((actual_labels - pred_labels) ** 2)

    # Total Sum of Squares (TSS)
    tss = torch.sum((actual_labels - mean_actual) ** 2)

    # Calculate R² score
    r2_score = 1 - sse / tss

    return r2_score

In [ ]:
def train_model(model, loss_fun, optimizer, train_loader, nepochs, hidden_layers, hidden_dim):
    losses = np.zeros(nepochs)
    r2_scores = np.zeros(nepochs)

    for epoch in range(nepochs):
        batch_losses = []
        batch_scores = []

        model.train()

        for inputs, labels in iter(train_loader):
            current_batch_size = labels.size(0)
            # Initialize the hidden state (h0, c0) for each batch
            h0 = torch.zeros(hidden_layers, current_batch_size, hidden_dim)
            c0 = torch.zeros(hidden_layers, current_batch_size, hidden_dim)
            h = (h0, c0)

            # Forward pass through the model
            y_hat, h_c_output = model(inputs, h, current_batch_size)

            # Zero the gradients
            optimizer.zero_grad()

            # Calculate loss and backpropagate
            loss = loss_fun(y_hat, labels.float())  #  y_hat.squeeze() might cause issues if the output tensor has a single dimension
            loss.backward()
            batch_losses.append(loss.item())

            # Update model parameters
            optimizer.step()

            # Calculate R² score for this batch
            r_score = calculate_r2_score(labels, y_hat)
            batch_scores.append(r_score.item())  # Ensure it's converted to a Python scalar

        # Average losses and R² scores across all batches for this epoch
        losses[epoch] = np.mean(batch_losses)
        r2_scores[epoch] = np.mean(batch_scores)

    return losses, r2_scores


In [ ]:
nepochs = 20
losses, r_score = train_model(model, loss_fun, optimizer, train_loader, nepochs, hidden_layers, batch_size, hidden_dim)

In [ ]:
first_batch = next(iter(train_loader))
inputs, labels = first_batch

print(inputs)
print(labels)


In [ ]:
??nn.LSTM

In [ ]:
# !pip install --upgrade torch
!pip install nltk

In [ ]:
import nltk

# Print out the list of directories NLTK searches for data
print(nltk.data.path)


In [ ]:
# List the contents of the 'wordnet' directory
# wordnet_dir = '/kaggle/input/wordnet/wordnet'
# print("Contents of wordnet directory:", os.listdir(wordnet_dir))


In [ ]:
import os

# Print the current working directory
# print(f"Current working directory: {os.getcwd()}")


In [ ]:
# import shutil

# # Paths
# source_dir = '/kaggle/input/wordnet/wordnet'  # Path where files are currently located
# destination_dir = '/kaggle/working/nltk_data/corpora/wordnet'  # Target directory

# # Ensure the destination directory exists
# os.makedirs(destination_dir, exist_ok=True)

# # Move files
# for item in os.listdir(source_dir):
#     source_path = os.path.join(source_dir, item)
#     destination_path = os.path.join(destination_dir, item)
#     if os.path.isdir(source_path):
#         shutil.copytree(source_path, destination_path, dirs_exist_ok=True)
#     else:
#         shutil.copy2(source_path, destination_path)

# print("Files moved successfully.")


In [ ]:
# import nltk

# # Add the extraction path to NLTK's data path
# nltk.data.path.append('/kaggle/working/nltk_data')

# # Print out the NLTK data paths
# print("NLTK data paths:", nltk.data.path)

# # Check if 'wordnet' is available
# try:
#     nltk.data.find('corpora/wordnet')
#     print("WordNet is installed and found.")
# except LookupError:
#     print("WordNet is not installed.")


In [ ]:
# import nltk

# # Download WordNet directly
# nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')

# print("WordNet download complete.")
